In [10]:
import getpass
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.model_selection import GridSearchCV #cross validation
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import plot_confusion_matrix
import datetime
import pyodbc
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [12]:
teamproductyear_id = 101

In [13]:
# connect to SQL Server.
server = '52.44.171.130' 
database = 'datascience' 
username = getpass.getpass(prompt='Enter your username') 
password = getpass.getpass(prompt='Enter your password')
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
query = f'''select teamproductyearid,lkupclientid,clientcode,productgrouping,trainseasonyear,testseasonyear,facttestyear,facttestprevyear,stlrDBName from ds.productyear_all r where teamproductyearid ={teamproductyear_id} ;'''
dfparam3 = pd.read_sql(query, cnxn)
cnxn.commit()
cursor.close()

In [14]:
dfparam3.head()

,teamproductyearid,lkupclientid,clientcode,productgrouping,trainseasonyear,testseasonyear,facttestyear,facttestprevyear,stlrDBName
0,101,93,nhlpanthers,Full Season,2019,2021,2021,None,stlrNHLPanthers


In [15]:
paramindex= 0

In [16]:
client_id = dfparam3._get_value(0,'lkupclientid')
client_code= dfparam3._get_value(0,'clientcode')
product_grouping =dfparam3._get_value(0,'productgrouping') 
train_season_year =dfparam3._get_value(0,'trainseasonyear') 
test_season_year =dfparam3._get_value(0,'testseasonyear') 
stlrdbname= dfparam3._get_value(0,'stlrDBName') 
fact_test_year= dfparam3._get_value(0,'facttestyear')

In [17]:
# connect to SQL Server.
database = stlrdbname 
cnxn2 = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor2 = cnxn2.cursor()
# Prepare the stored procedure execution script and parameter values
storedProc = f'''Exec [ds].[getRetentionScoringModelData] {client_id}'''
params = (client_id)
cnxn2.commit()
cursor2.close()

In [18]:
dftrain = pd.read_sql(storedProc, cnxn2)

In [19]:
dftrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12708 entries, 0 to 12707
Data columns (total 55 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   lkupClientId              12708 non-null  int64  
 1   dimCustomerMasterId       12708 non-null  int64  
 2   customerNumber            12708 non-null  object 
 3   year                      12708 non-null  object 
 4   productGrouping           12708 non-null  object 
 5   totalSpent                12708 non-null  float64
 6   recentDate                12708 non-null  object 
 7   attendancePercent         12708 non-null  float64
 8   renewedBeforeDays         12708 non-null  int64  
 9   isBuyer                   12708 non-null  object 
 10  source_tenure             12708 non-null  int64  
 11  tenure                    12708 non-null  int64  
 12  distToVenue               0 non-null      object 
 13  totalGames                12708 non-null  int64  
 14  recenc

In [11]:
dfproduct = dftrain[dftrain["productGrouping"] ==  product_grouping ]

,lkupClientId,dimCustomerMasterId,customerNumber,year,productGrouping,totalSpent,recentDate,attendancePercent,renewedBeforeDays,isBuyer,...,lengthOfResidenceInYrs,annualHHIncome,education,urbanicity,isnextyear_buyer,isnextyear_samepkg_buyer,pkgupgrade_status,auto_renewal,credits_after_refund,isNextYear_Buyer
2,11,272817560,100804773,2012,Full Season,2000.0,2012-08-31,0.524390,28,TRUE,...,None,None,None,None,None,None,None,None,0.0,0
4,11,272848152,100973984,2012,Full Season,2056.0,2012-06-12,0.035573,28,TRUE,...,None,None,None,None,None,None,None,None,0.0,0
5,11,272849830,100907090,2012,Full Season,784.0,2012-08-26,0.693878,27,TRUE,...,None,None,None,None,None,None,None,None,0.0,0
11,11,272930600,103097042,2012,Full Season,560.0,1970-01-01,0.000000,28,TRUE,...,None,None,None,None,None,None,None,None,0.0,0
19,11,272871159,103795062,2012,Full Season,880.0,2012-08-26,0.854545,185,TRUE,...,None,None,None,None,None,None,None,None,0.0,0


In [10]:
dfproduct['year']= pd.to_numeric(dfproduct['year'])

In [11]:
dfyear = dfproduct[dfproduct["year"] < train_season_year]

In [12]:
df3 = dfyear[["dimCustomerMasterId", "recency","attendancePercent","totalSpent","distToVenue","source_tenure","renewedBeforeDays","missed_games_1","missed_games_2","missed_games_over_2","isNextYear_Buyer"]]
df3.head()

,dimCustomerMasterId,recency,attendancePercent,totalSpent,distToVenue,source_tenure,renewedBeforeDays,missed_games_1,missed_games_2,missed_games_over_2,isNextYear_Buyer
2,272817560,0,0.524390,2000.0,31.35,177,28,6,5,7,0
4,272848152,30,0.035573,2056.0,20.53,177,28,0,0,1,0
5,272849830,1,0.693878,784.0,27.90,175,27,6,1,2,0
11,272930600,0,0.000000,560.0,32.93,177,28,0,0,1,0
19,272871159,0,0.854545,880.0,30.64,328,185,2,0,2,0


In [14]:
df3['dimCustomerMasterId']= pd.to_numeric(df3['dimCustomerMasterId'])
df3['attendancePercent']= pd.to_numeric(df3['attendancePercent'])
df3['totalSpent']= pd.to_numeric(df3['totalSpent'])
df3['distToVenue']= pd.to_numeric(df3['distToVenue'])

In [17]:
X = df3.drop(['isNextYear_Buyer'], axis=1).copy()
y = df3['isNextYear_Buyer'].copy()

In [18]:
clf= xgb.XGBClassifier(objective='binary:logistic',seed=42,gamma=0.25,lear_rate=0.1,max_depth=6,reg_lambda=20,scale_pos_weight=3,subsample=0.9,colsample_bytree=0.5)
clf.fit(X,y,verbose=True,early_stopping_rounds=10,eval_metric='aucpr',eval_set=[(X,y)])

[12:58:44] WARNING: ../src/learner.cc:541: 
Parameters: { lear_rate } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation_0-aucpr:0.79168
[1]	validation_0-aucpr:0.84152
[2]	validation_0-aucpr:0.84988
[3]	validation_0-aucpr:0.90509
[4]	validation_0-aucpr:0.91117
[5]	validation_0-aucpr:0.91736
[6]	validation_0-aucpr:0.92527
[7]	validation_0-aucpr:0.92774
[8]	validation_0-aucpr:0.92975
[9]	validation_0-aucpr:0.93160
[10]	validation_0-aucpr:0.93300
[11]	validation_0-aucpr:0.94147
[12]	validation_0-aucpr:0.95239
[13]	validation_0-aucpr:0.95317
[14]	validation_0-aucpr:0.95503
[15]	validation_0-aucpr:0.95531
[16]	validation_0-aucpr:0.95918
[17]	validation_0-aucpr:0.96071
[18]	validation_0-aucpr:0.96336
[19]	validation_0-aucpr:0.96481
[20]	validation_0-aucpr:0.96691
[21]	vali

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0.25, gpu_id=-1,
              importance_type='gain', interaction_constraints='', lear_rate=0.1,
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=42,
              reg_alpha=0, reg_lambda=20, scale_pos_weight=3, seed=42,
              subsample=0.9, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [21]:
# check Important features
feature_importances_df = pd.DataFrame(
    {"feature": list(X.columns), "importance": clf.feature_importances_}
).sort_values("importance", ascending=False)

In [22]:
feature_importances=feature_importances_df[['feature','importance']]
feature_importances['productgrouping'] = product_grouping
feature_importances=feature_importances[['feature','importance']]
feature_importances.drop([0],axis=0,inplace=True) 
feature_importances

,feature,importance
5,source_tenure,0.332997
3,totalSpent,0.079372
6,renewedBeforeDays,0.075125
2,attendancePercent,0.072197
7,missed_games_1,0.071847
1,recency,0.070956
4,distToVenue,0.065052
8,missed_games_2,0.059982
9,missed_games_over_2,0.056414


In [23]:
dftestyear = dfproduct[dfproduct["year"] >= test_season_year]

In [24]:
df_test = dftestyear[["dimCustomerMasterId", "recency","attendancePercent","totalSpent","distToVenue","source_tenure","renewedBeforeDays","missed_games_1","missed_games_2","missed_games_over_2","isNextYear_Buyer"]]

In [25]:
df_test.count()

dimCustomerMasterId    209
recency                209
attendancePercent      209
totalSpent             209
distToVenue            209
source_tenure          209
renewedBeforeDays      209
missed_games_1         209
missed_games_2         209
missed_games_over_2    209
isNextYear_Buyer       209
dtype: int64

In [26]:
df_test['dimCustomerMasterId']= pd.to_numeric(df_test['dimCustomerMasterId'])
df_test['attendancePercent']= pd.to_numeric(df_test['attendancePercent'])
df_test['totalSpent']= pd.to_numeric(df_test['totalSpent'])
df_test['distToVenue']= pd.to_numeric(df_test['distToVenue'])

In [27]:
X_test = df_test.drop(['isNextYear_Buyer'], axis=1).copy()

In [28]:
X_test.reset_index(drop=True, inplace=True)

In [29]:
y_pred = clf.predict_proba(X)

In [30]:
# make predictions for test data
y_pred_test = clf.predict_proba(X_test)

In [31]:
# Creating the array to convert
array_y_pred_test = np.array(y_pred_test)

In [32]:
# Create the dataframe
df_y_pred_test = pd.DataFrame(array_y_pred_test)
df_y_pred_test.columns = ['nonbuyer','buyer']

In [33]:
result_test = pd.concat([df_y_pred_test, X_test], axis=1, join="inner")

In [34]:
result_test = result_test.drop(['nonbuyer'], axis=1).copy()

In [35]:
result_test['buyer']= pd.to_numeric(result_test['buyer'])

In [36]:
merged_inner = pd.merge(left=result_test, right=dftestyear, how='inner', left_on='dimCustomerMasterId', right_on='dimCustomerMasterId')

In [37]:
today = datetime.datetime.now()
date_time = today.strftime("%m-%d-%Y %H:%M:%S")

In [38]:
newscors=merged_inner[['dimCustomerMasterId','buyer','source_tenure_x','attendancePercent_x','recentDate']]
newscors.columns = ['dimCustomerMasterId','buyer_score','tenuredays','attendancePercentage','mostrecentattendance']
newscors['year'] = test_season_year
newscors['lkupclientid'] = client_id
newscors['productgrouping'] = product_grouping
newscors['currVersnFlag']= 1
newscors['loadid']= 0
newscors['seasonyear']= fact_test_year
newscors['insertDate'] = date_time

In [39]:
# connect to SQL Server.
database = 'datascience' 
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
# Insert Dataframe into SQL Server:
for index, row in newscors.iterrows():
    cursor.execute("INSERT INTO ds.customerScores (dimCustomerMasterId,buyer_score,tenuredays,attendancePercentage,mostrecentattendance,year,lkupclientid,productgrouping,seasonYear,insertDate) values(" + str(row.dimCustomerMasterId) + "," + str(round(row.buyer_score,4))+ ","+ str(row.tenuredays)+ ","+ str(row.attendancePercentage) + "," +"'"+ str(row.mostrecentattendance)+"'" + ","+ str(row.year) + "," + str(row.lkupclientid)+ ","+"'"+str(row.productgrouping)+"'"+","+str(fact_test_year)+"," +"'"+str(row.insertDate)+"'" + ")")
cnxn.commit()
cursor.close()

In [40]:
feature_importances2= feature_importances

In [41]:
feature_importances2.at[1,'feature']='Recency'
feature_importances2.at[2,'feature']='Attendance'
feature_importances2.at[3,'feature']='Monetary'
feature_importances2.at[4,'feature']='Distance to Venue'
feature_importances2.at[5,'feature']='Tenure'
feature_importances2.at[6,'feature']='Time to Renew'
feature_importances2.at[7,'feature']='Missed Games Streak 1'
feature_importances2.at[8,'feature']='Missed Games Streak 2'
feature_importances2.at[9,'feature']='Missed Games Streak Over 2'

In [42]:
feature_importances2['attrank']={1,2,3,4,5,6,7,8,9}
feature_importances2 ['lkupClientId'] = client_id
feature_importances2 ['modelVersnNumber'] = 2
feature_importances2 ['scoreDate'] = date_time
feature_importances2 ['loadId'] = 0
feature_importances2 ['product'] = product_grouping
feature_importances2.columns=['attribute','indexValue','attrank','lkupClientId','modelVersnNumber','scoreDate','loadId','product']

In [43]:
# connect to SQL Server.
database = stlrdbname 
# connect to SQL Server. 
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
# Insert Dataframe into SQL Server:
for index, row in feature_importances2.iterrows():
    cursor.execute("INSERT INTO dw.lkupRetentionAttributeImportance (attribute,product,indexValue,rank,lkupClientId,modelVersnNumber,scoreDate,loadId) values(" + "'" +str(row.attribute)+"'"+","+ "'"+str(product_grouping)+"'" +"," + str(round(row.indexValue,4)) + "," + str(row.attrank)+ ","+ str(row.lkupClientId) + "," + str(row.modelVersnNumber)+ "," + "'"+ str(row.scoreDate)+ "'"+ ","+ str(row.loadId)  + ")")
cnxn.commit()
cursor.close()